In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
import pandas_profiling
import missingno as msno
import datetime
import re

# Read data

## Twitter data

In [2]:
twitter1 = pd.read_csv("trayvontweets/Trayvon Tweets 2012.csv",encoding = "latin1",index_col = 0)

/Users/ycr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ycr/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
twitter2 = pd.read_csv("trayvontweets/Trayvon Tweets 2013.csv",encoding = "latin1", index_col = 0)

In [4]:
def date(dates):
    delta=datetime.timedelta(days=dates)
    today=datetime.datetime.strptime('1899-12-30 00:00:00','%Y-%m-%d %H:%M:%S')+delta
    return datetime.datetime.strftime(today,'%Y-%m-%d %H:%M:%S')

In [5]:
twitter = pd.concat([twitter1,twitter2],ignore_index=True)

In [6]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097150 entries, 0 to 2097149
Data columns (total 20 columns):
id                      int64
message_id              int64
user_handle             object
orig_user_id            int64
orig_user_handle        object
topsy_type              object
orig_date_posted_gmt    float64
date_posted_gmt         float64
message                 object
orig_user_location      object
coordinates             object
tweet_location          object
longitude               float64
latitude                float64
created_at_gmt          float64
seconds_after_event     float64
FIPS                    object
County                  object
City                    object
State                   object
dtypes: float64(6), int64(3), object(11)
memory usage: 320.0+ MB


-----

In [7]:
twitter = twitter.sample(n = 1000)
twitter = twitter.reset_index()

In [8]:
twitter.loc[:,'message'] = twitter.loc[:,'message'].fillna('nan')

In [9]:
twitter = twitter.drop('index',axis = 1)

In [10]:
twitter.loc[:,'topsy_type'].unique()

array(['retweet:native', 'tweet', 'retweet:reply'], dtype=object)

In [11]:
# uncomment to output twitter message as txt
#twitter_message = pd.DataFrame(twitter_message)
#twitter_message = twitter_message.reset_index()
#twitter_message.to_csv("twitter_message",sep = "\t",index=False, header=False)

In [12]:
import re
from functools import partial
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [13]:
def removeUnicode(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
    text = re.sub(r'[^\x00-\x7f]',r'',text)
    return text

In [14]:
def replaceURL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(http?://[^\s]+))','url',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

In [15]:
def replaceAtUser(text):
    """ Replaces "@user" with "atUser" """
    text = re.sub('@[^\s]+','atUser',text)   #keep atUser or delete?
    return text

In [16]:
def removeHashtagInFrontOfWord(text):
    """ Removes hastag in front of a word """
    text = re.sub(r'#([^\s]+)', r'\1', text)  #only remove# or remove all?
    return text

In [17]:
def removeRT(text):
    """ Removes hastag in front of a word """
    text = re.sub(r'\bRT\b', r'', text)
    return text

In [18]:
with open('slang.txt') as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2]) for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True) # longest first for regex
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])

In [26]:
#spell correction
def words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('corporaForSpellCorrection.txt').read()))

def P(word, N=sum(WORDS.values())): 
    """P robability of `word`. """
    return WORDS[word] / N

def spellCorrection(word): 
    """ Most probable spelling correction for word. """
    return max(candidates(word), key=P)

def candidates(word): 
    """ Generate possible spelling corrections for word. """
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    """ The subset of `words` that appear in the dictionary of WORDS. """
    return set(w for w in words if w in WORDS)

def edits1(word):
    """ All edits that are one edit away from `word`. """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    """ All edits that are two edits away from `word`. """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [24]:
####### STOPS HERE
#use bag model or sequence model?
def replace(word, pos=None):
    """ Creates a set of all antonyms for the word and if there is only one antonym, it returns it """
    antonyms = set()
    for syn in wordnet.synsets(word, pos=pos):
        for lemma in syn.lemmas():
            for antonym in lemma.antonyms():
                antonyms.add(antonym.name())
    if len(antonyms) == 1:
        return antonyms.pop()
    else:
        return None

def replaceNegations(text):
    """ Finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym """
    i, l = 0, len(text)
    words = []
    while i < l:
        word = text[i]
        if word == 'not' and i+1 < l:
            ant = replace(text[i+1])
            if ant:
                words.append(ant)
                i += 2
                continue

        words.append(word)
        i += 1
    return words

In [22]:
for i in range(twitter.shape[0]):
    twitter.loc[i,'message_cleaned'] = removeUnicode(twitter.loc[i,'message'])
    twitter.loc[i,'message_cleaned'] = replaceURL(twitter.loc[i,'message_cleaned'])
    twitter.loc[i,'message_cleaned'] = replaceAtUser(twitter.loc[i,'message_cleaned'])    
    twitter.loc[i,'message_cleaned'] = removeHashtagInFrontOfWord(twitter.loc[i,'message_cleaned'])
    twitter.loc[i,'message_cleaned'] = removeRT(twitter.loc[i,'message_cleaned'])
    twitter.loc[i,'message_cleaned'] = replaceSlang(twitter.loc[i,'message_cleaned'])

In [23]:
twitter.loc[:,["message","message_cleaned"]]

,message,message_cleaned
0,@WhiskersOnKtns Trayvon was profiled by Zimmer...,atUser Trayvon was profiled by Zimmerman and t...
1,my pastor just tied Trayvon Martin into his se...,my pastor just tied Trayvon Martin into his se...
2,People off the chain on the internet ain't the...,People off the chain on the internet ain't the...
3,Shellie Zimmerman & George Zimmerman Divorce ä...,Shellie Zimmerman & George Zimmerman Divorce ...
4,Zimmerman lawyer showed prosecution weaknesses...,Zimmerman lawyer showed prosecution weaknesses...
5,Full video: President Obama speaks on Trayvon ...,Full video: President Obama speaks on Trayvon ...
6,Muhfuckas was jus on that stop Kony shit a wee...,Muhfuckas was jus on that stop Kony shit a wee...
7,President Clinton Hopes Trayvon Case--Reapprai...,President Clinton Hopes Trayvon Case--Reapprai...
8,If my parents do not shutup about this Trayvon...,If my parents do not shutup about this Trayvon...
9,Look how innocent Trayvon Martin looks http://...,Look how innocent Trayvon Martin looks url


- nan in message:      
is it all "reply"?
- clean words/spell correction:       
yess, sooooooo
- special character &:    
"As we remember Trayvon Martin in our worship& sermons let us revisit Dr. King's eulogy4 the young people who died in the Birmingham Church!"     
"With this Trayvon verdict & Fruitvale Station movie coming out hopefully we can start to open USA's eyes to what's going on in our country"
- special character !:    
multiple ones
- special character ?:    
multiple ones 
- negation
- get all hashtags